## Comparable score ranges experiment - *easy* discrimination task
The following notebook explores the comparability of score ranges for the *easy* task when using **non-overlapping** signatures for **three** available cell types (B cells, Monocytes, NK cells).

After selecting the cell type specific signatures we score with each scoring method the signatures and apply hard labeling on the scores as well as on the probabilities returned by the GMM postprocessing.

This jupyter notebook uses the data and differentially expressed genes found [here](https://atlas.fredhutch.org/nygc/multimodal-pbmc/).

In [ ]:
import itertools
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from statannotations.Annotator import Annotator
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, jaccard_score, balanced_accuracy_score

sys.path.append('../../..')
from data.load_data import load_datasets
from data.constants import BASE_PATH_EXPERIMENTS, BASE_PATH_DATA

from signaturescoring import score_signature
from signaturescoring.scoring_methods.gmm_postprocessing import GMMPostprocessor
from signaturescoring.utils.utils import check_signature_genes, get_mean_and_variance_gene_expression

sc.settings.verbosity = 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
plt.rcParams.update({'pdf.fonttype':42, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'font.size':14})

## Global variables

In [ ]:
## define the path where the PBMC data is stored
dataset = 'pbmc_b_mono_nk'
norm_method = 'mean'
DE_of_celltypes_fn = os.path.join(BASE_PATH_DATA, 'annotations/citeseq_pbmc/DE_by_celltype.csv')

In [ ]:
## define the path where data should be stored.
storing_path = os.path.join(BASE_PATH_EXPERIMENTS, 'comparable_score_ranges/B_NK_Mono/scoring_all_overlapping_signatures')
if not os.path.exists(storing_path):
    os.makedirs(storing_path)
    sc.logging.info(f'Created new directory with path {storing_path}')

In [ ]:
save = True

## Load preprocessed data

In [ ]:
adata = load_datasets(dataset, norm_method=norm_method)
if 'log1p' in adata.uns_keys():
    adata.uns['log1p']['base'] = None
else:
    adata.uns['log1p'] = {'base': None}

In [ ]:
adata.obs['celltype.l1'].value_counts()

### Get for each celltype in level 1 the set of types in level 3 and the same for celltype level 2

In [ ]:
celltype_sets_l2 = {}
for group in  adata.obs[['celltype.l2','celltype.l3']].groupby(by='celltype.l2'):
    celltype_sets_l2[group[0]] = list(np.unique(group[1]['celltype.l3']))

In [ ]:
celltype_sets_l1 = {}
for group in  adata.obs[['celltype.l1','celltype.l2']].groupby(by='celltype.l1'):
    new_celltype_list = []
    for celltype in list(np.unique(group[1]['celltype.l2'])):
        new_celltype_list.append(celltype_sets_l2[celltype])
    celltype_sets_l1[group[0]] = list(itertools.chain(*new_celltype_list))


In [ ]:
celltype_sets_l1

### Look at the differentially expressed genes given by the paper
The differential gene expression is done on level 3 celltypes. The logfoldchanges for the genes of different cell types are not comparable.

In [ ]:
## define path to table with DGEX genes
DE_of_celltypes = pd.read_csv(DE_of_celltypes_fn)

get signatures for celltype level 1

In [ ]:
grouped_DE_of_celltypes = DE_of_celltypes.groupby(by='Cell Type')

In [ ]:
SG_subtypes = {}
for key, subtypes in celltype_sets_l1.items():
    sig_genes = set()
    for subtype in subtypes:
        if subtype not in grouped_DE_of_celltypes.groups.keys():
            continue
        group = grouped_DE_of_celltypes.get_group(subtype)
        sig_genes.update(group.sort_values(by='Average Log Fold Change', ascending=False)['Gene'].iloc[0:300])
    SG_subtypes[key] = sig_genes

In [ ]:
SG_subtypes.keys()

In [ ]:
for key, val in SG_subtypes.items():
    print(f'signature for subtype {key} has length {len(val)}')

Remove signature genes with too high average expression

In [ ]:
for key, val in SG_subtypes.items():
    print(f'signature for {key} has length {len(val)}')
    SG_subtypes[key] = list(val)
    SG_subtypes[key]  = check_signature_genes(adata.var_names, val)

In [ ]:
plt.rcParams.update({'pdf.fonttype':42, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'font.size':14})
df_mean_var = get_mean_and_variance_gene_expression(
    adata,
    estim_var=True,
    show_plots=True,
    #store_path=storing_path,
    #store_data_prefix='all'
)

In [ ]:
plt.rcParams.update({'pdf.fonttype':42, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'font.size':14})

In [ ]:
for k,v in SG_subtypes.items():
    print(f'Signature for subtype {k} contains {len(v)} genes.')
    SG_subtypes[k] = list(v)
    plt.figure(figsize=(10,10))
    allowed_v = [] 
    plt.plot(df_mean_var['mean'].values)
    for sig_gene in v:
        sig_gene_idx = np.argwhere(df_mean_var['mean'].index ==sig_gene)[0]
        
        if sig_gene_idx<= (df_mean_var.shape[0]-50):
            plt.axvline(sig_gene_idx,c='g')
            allowed_v.append(sig_gene)
        else:
            plt.axvline(sig_gene_idx,c='r')
    SG_subtypes[k] = allowed_v  
    plt.xlim([df_mean_var.shape[0]-100,df_mean_var.shape[0]+50])
    plt.title(f'avg. expression signature genes for {k}', fontsize=16)
    if save:
        path = os.path.join(storing_path, 'mean_expr_genes')
        if not os.path.exists(path):
            os.makedirs(path)
            sc.logging.info(f'Created new directory with path {path}')
        plt.savefig(os.path.join(path, f'{k}.png'), format = 'png')
    else:
        print('not storing image')
    plt.show()

In [ ]:
for k,v in SG_subtypes.items():
    print(f'Signature for subtype {k} contains {len(v)} genes.')

### Use for all signatures the same gene pool create 

In [ ]:
all_sig_genes = set() 
for key, val in SG_subtypes.items():
    all_sig_genes.update(val)

In [ ]:
gene_pool = list(set(adata.var_names).difference(all_sig_genes))

### Score  marker genes (differentially expressed genes) for specifc celltypes of level 1 given by the paper

In [ ]:
n_bins = 25
n_ctrl_genes = 100

In [ ]:
scoring_methods = [
    {
        "scoring_method": "scanpy_scoring",
        "sc_params": {
            "ctrl_size": n_ctrl_genes,
            "n_bins": n_bins,
            "score_name": "Scanpy",
        },
    },
    {
        "scoring_method": "seurat_scoring",
        "sc_params": {
            "ctrl_size": n_ctrl_genes,
            "n_bins": n_bins,
            "score_name": "Seurat",
            "gene_pool":gene_pool
        },
    },
    {
        "scoring_method": "adjusted_neighborhood_scoring",
        "sc_params": {
            "ctrl_size": n_ctrl_genes,
            "score_name": "ANS",
            "gene_pool":gene_pool
        },
    },
    {
        "scoring_method": "seurat_ag_scoring",
        "sc_params": {
            "n_bins": n_bins,
            "score_name": "Seurat_AG",
            "gene_pool":gene_pool
        },
    },
    {
        "scoring_method": "seurat_lvg_scoring",
        "sc_params": {
            "ctrl_size": n_ctrl_genes,
            "n_bins": n_bins,
            "lvg_computation_version": "v1",
            "lvg_computation_method": "seurat",
            "score_name": "Seurat_LVG",
            "gene_pool":gene_pool
        },
    },
    {
        "scoring_method": "ucell_scoring",
        "sc_params": {
            "score_name": "UCell",
        },
    },
    {
        "scoring_method": "jasmine_scoring",
        "sc_params": {
            "score_method": 'likelihood',
            "score_name": "Jasmine_LH",
        },
    },
    {
        "scoring_method": "jasmine_scoring",
        "sc_params": {
            "score_method": 'oddsratio',
            "score_name": "Jasmine_OR",
        },
    },
]

In [ ]:
method_wo_mean = ['scanpy_scoring', 'corrected_scanpy_scoring','ucell_scoring','jasmine_scoring']

In [ ]:
sc_names = ['ANS', 'Seurat', 'Seurat_AG', 'Seurat_LVG', 'Scanpy', 'Jasmine_LH', 'Jasmine_OR', 'UCell']

In [ ]:
len(set(df_mean_var.index).difference(set(gene_pool)))

In [ ]:
scoring_names = []
for sc_method in scoring_methods:
    
    scoring_method = sc_method['scoring_method']
    sc_params = sc_method['sc_params']
    
    print(f'Running scoring with scoring method {scoring_method}')
    
    for k1, v1 in SG_subtypes.items():
        
        print(f'   > Running scoring for signatures of celltyple-l1 {k1}')
        
        curr_sc_params = sc_params.copy()
        curr_sc_params['score_name'] = curr_sc_params['score_name'] +'_'+k1

        if scoring_method in method_wo_mean:
            score_signature(method=scoring_method,
                            adata=adata,
                            gene_list=v1,
                            **curr_sc_params)
        else:
            score_signature(method=scoring_method,
                        adata=adata,
                        gene_list=v1,
                        df_mean_var=df_mean_var,
                        **curr_sc_params)
        scoring_names.append(curr_sc_params['score_name'])
            

In [ ]:
scoring_names = [x for x in adata.obs.columns if any([y in sc_names or y == 'Jasmine' for y in x.split('_')])]
scoring_names

In [ ]:
for i in range(0, len(scoring_names), 3):
    gmm_post = GMMPostprocessor(
        n_components=3
    )
    
    store_name_pred, store_names_proba, _ = gmm_post.fit_and_predict(adata, scoring_names[i:(i+3)])
    assignments = gmm_post.assign_clusters_to_signatures(adata, scoring_names[i:(i+3)], store_names_proba, plot=False)
    print(assignments)
    for key, val in assignments.items():
        adata.obs[key+'_gmm_3K'] = adata.obs[val].copy()

In [ ]:
adata.obs = adata.obs.drop(columns = [x for x in adata.obs.columns if ('_GMM_proba' in x) or ('_GMM_pred' in x)])

In [ ]:
scoring_names = [x for x in adata.obs.columns if any([y in sc_names or y == 'Jasmine' for y in x.split('_')])]
scoring_names.sort()

In [ ]:
scoring_names

In [ ]:
score_name_wo_gmm = [x for x in scoring_names if 'gmm' not in x]
score_name_w_gmm = [x for x in scoring_names if 'gmm_3K' in x]
score_name_wo_gmm = sorted(score_name_wo_gmm, key=lambda x: x.rsplit('_', 1)[0])
score_name_w_gmm = sorted(score_name_w_gmm, key=lambda x: x.rsplit('_', 3)[0])
score_name_wo_gmm, score_name_w_gmm

### evaluate scores

In [ ]:
tmp = adata.obs[['celltype.l1']+score_name_wo_gmm]

In [ ]:
tmp = tmp.melt(id_vars=['celltype.l1'],
        var_name='scoring_method',
        value_name='scores')
tmp

In [ ]:
tmp.scoring_method.value_counts().sort_index()

In [ ]:
tmp['scoring_method_short'] = tmp.scoring_method.apply(lambda x: '_'.join(x.split('_')[0:-1]))

In [ ]:
tmp['Scoring for signature'] = tmp.scoring_method.apply(lambda x: x.split('_')[-1])

In [ ]:
tmp['scoring_method_short'].value_counts()

In [ ]:
if save:
    fn = os.path.join(storing_path, 'data_for_violin_plot_normal_scores.csv')
    tmp.to_csv(fn)
    sc.logging.info(f'Storing obs scores at {fn}')

In [ ]:
order=['B', 'Mono', 'NK']

In [ ]:
yticks = [-1, -0.5, 0, 0.5, 1.0]

yticks = [round(x,2) for x in yticks]

In [ ]:
g = sns.catplot(data=tmp,
                x='celltype.l1', 
                y='scores', 
                hue='Scoring for signature', 
                col_order=sc_names,
                col= 'scoring_method_short',  
                kind='violin', 
                col_wrap=4,
                order=order,
                legend=False
               )
g.set_ylabels('Scores', size=22)
g.set_titles("{col_name}", size=24)
g.set_xticklabels(order, size=22)
g.set(xlabel=None)
g.fig.subplots_adjust(top=0.88)
g.fig.suptitle('$\it{Easy}$ task', fontsize=26)
g.add_legend(fontsize=22, title='Signature')
g.legend.get_title().set_fontsize(22)
g.set(yticks=yticks)
g.set_yticklabels(yticks, size=20)
if save:
    plt.savefig(os.path.join(storing_path, 'violin_plots_not_comparable_ranges.svg'), format='svg')

In [ ]:
gt = adata.obs['celltype.l1'].copy()
gt

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(adata.obs[['celltype.l1']]).toarray())
enc_df

In [ ]:
rows = []
for i in range(0,len(score_name_wo_gmm),3):
    
    prediction = adata.obs[score_name_wo_gmm[i:(i+3)]]
    
#     curr_auc = roc_auc_score(enc_df, prediction, average='weighted')
    tmp = prediction.idxmax(axis=1)
    tmp[tmp.str.contains('B')] = 'B'
    tmp[tmp.str.contains('NK')] = 'NK'
    tmp[tmp.str.contains('Mono')] = 'Mono'
 
    curr_f1 = f1_score(gt,tmp, average='weighted')
    curr_j = jaccard_score(gt,tmp, average='weighted')
    curr_ba = balanced_accuracy_score(gt,tmp)
    
    row = {
        'Scoring method': '_'.join(score_name_wo_gmm[i].split('_')[0:-1]),
#         'AUCROC (weighted)':curr_auc,
        'F1-score (weighted)':curr_f1, 
        'Jaccard-score (weighted)':curr_j,
        'Balanced accuracy':curr_ba
    }
    rows.append(row)

In [ ]:
performance_hard_labeling_on_scores = pd.DataFrame(rows)

performance_hard_labeling_on_scores

In [ ]:
if save:
    performance_hard_labeling_on_scores.to_csv(os.path.join(storing_path, 'performance_hard_labeling_on_scores.csv'))

In [ ]:
performance_hard_labeling_on_scores = performance_hard_labeling_on_scores.melt(id_vars=['Scoring method', 'F1-score (weighted)'],
            var_name='metric',
            value_name='met_score'
)

f = plt.figure(figsize=(8, 6))
g = sns.scatterplot(
    x='met_score',
    y='F1-score (weighted)',
    hue='Scoring method',
    hue_order=sc_names,
    style='metric',
    data=performance_hard_labeling_on_scores,
    s=200
)
lgnd = g.legend(bbox_to_anchor=(1, 1), fontsize=16)
g.set_title('Performance hard labeling using scores ($\it{easy}$ task)', fontsize=18)
g.set_xlabel('Values of metrics', fontsize=16)
g.set_ylabel('F1-score (weighted)', fontsize=16)
if save:
    f.savefig(os.path.join(storing_path, f'scores_hard_labeling.svg'), format='svg')
    f.savefig(os.path.join(storing_path, f'scores_hard_labeling.png'), format='png', dpi=300)

### evaluate GMM outcome

In [ ]:
tmp = adata.obs[['celltype.l1']+score_name_w_gmm]

In [ ]:
tmp = tmp.melt(id_vars=['celltype.l1'],
        var_name='scoring_method',
        value_name='scores')
tmp

In [ ]:
tmp['scoring_method_short'] = tmp.scoring_method.apply(lambda x: '_'.join(x.split('_')[0:-3])+' with GMM 3K')

In [ ]:
tmp['Scoring for signature'] = tmp.scoring_method.apply(lambda x: x.split('_')[-3])

In [ ]:
tmp['scoring_method_short'].value_counts()

In [ ]:
tmp = tmp[tmp.scoring_method_short.str.contains('std_adjust')==False]

In [ ]:
tmp['scoring_method_short'].value_counts()

In [ ]:
if save:
    tmp.to_csv(os.path.join(storing_path, 'data_for_violin_plot_gmm_scores.csv'))

In [ ]:
order=['B', 'Mono', 'NK']
yticks = [0, 0.2,0.4,0.6,0.8,1.0]
sc_names = ['ANS with GMM 3K', 
            'Seurat with GMM 3K', 
            'Seurat_AG with GMM 3K',
            'Seurat_LVG with GMM 3K', 
            'Scanpy with GMM 3K', 
            'Jasmine_LH with GMM 3K',
            'Jasmine_OR with GMM 3K',
            'UCell with GMM 3K']

In [ ]:
g = sns.catplot(data=tmp[tmp['scoring_method_short'].str.contains('var adjustment')==False],
                x='celltype.l1', 
                y='scores', 
                hue='Scoring for signature', 
                col= 'scoring_method_short', 
                col_order=sc_names, 
                kind='violin',
                order=order,
                #height=10, 
                #aspect=1
               )
g.set_ylabels('Scores', size=22)
g.set_titles("{col_name}", size=24)
g.set_xticklabels(order, size=22)
g.set(xlabel=None)
# g.add_legend(fontsize=14)
g.set(yticks=yticks)
g.set_yticklabels(yticks, size=20)
if save:
    plt.savefig(os.path.join(storing_path, 'violin_plots_not_comparable_ranges_GMM.svg'), format='svg')

In [ ]:
gt = adata.obs['celltype.l1'].copy()
gt

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(adata.obs[['celltype.l1']]).toarray())
enc_df

In [ ]:
score_name_w_gmm = [x for x in score_name_w_gmm if 'std_adjust' not in x]
score_name_w_gmm

In [ ]:
rows = []
for i in range(0,len(score_name_w_gmm),3):
    
    prediction = adata.obs[score_name_w_gmm[i:(i+3)]]
    
#     curr_auc = roc_auc_score(enc_df, prediction, average='weighted')
    
    tmp = prediction.idxmax(axis=1)
    tmp[tmp.str.contains('B')] = 'B'
    tmp[tmp.str.contains('NK')] = 'NK'
    tmp[tmp.str.contains('Mono')] = 'Mono'
 
    curr_f1 = f1_score(gt,tmp, average='weighted')
    curr_j = jaccard_score(gt,tmp, average='weighted')
    curr_ba = balanced_accuracy_score(gt,tmp)
    
    row = {
        'Scoring method': '_'.join(score_name_w_gmm[i].split('_')[0:-3])+' with GMM 3K',
        'F1-score (weighted)':curr_f1, 
        'Jaccard-score (weighted)':curr_j,
        'Balanced accuracy':curr_ba
    }
    rows.append(row)

In [ ]:
performance_hard_labeling_on_scores = pd.DataFrame(rows)

In [ ]:
performance_hard_labeling_on_scores.sort_values(by='Balanced accuracy', ascending=False)

In [ ]:
if save:
    performance_hard_labeling_on_scores.to_csv(os.path.join(storing_path, 'performance_hard_labeling_on_GMM.csv'))

In [ ]:
performance_hard_labeling_on_scores = performance_hard_labeling_on_scores.melt(id_vars=['Scoring method', 'F1-score (weighted)'],
            var_name='metric',
            value_name='met_value'
)

In [ ]:
f = plt.figure(figsize=(8, 6))
g = sns.scatterplot(
    x='met_value',
    y='F1-score (weighted)',
    hue='Scoring method',
    hue_order=sc_names,
    style='metric',
    data=performance_hard_labeling_on_scores,
    s=200
)
lgnd = g.legend(bbox_to_anchor=(1, 1), fontsize=16)
g.set_title('Performance hard labeling using probabilities ($\it{easy}$ task)', fontsize=18)
g.set_xlabel('Values of metrics', fontsize=16)
g.set_ylabel('F1-score (weighted)', fontsize=16)
if save:
    f.savefig(os.path.join(storing_path, f'GMM3_hard_labeling.svg'), format='svg')
    f.savefig(os.path.join(storing_path, f'GMM3_hard_labeling.png'), format='png', dpi=300)